In [ ]:
import cv2
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import argparse
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay, f1_score
import joblib
import torch
import torch.nn as nn
from torchvision.transforms import Resize, ToTensor, Compose, Normalize, transforms
from torchvision.models import resnet34, resnet18
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
data = pd.read_csv("/content/gdrive/MyDrive/NCKH/class.csv")

In [ ]:
data.head()

,Image,Class,Mean,Variance,Standard Deviation,Entropy,Skewness,Kurtosis,Contrast,Energy,ASM,Homogeneity,Dissimilarity,Correlation,Coarseness
0,Image1,0,6.535339,619.587845,24.891522,0.109059,4.276477,18.900575,98.613971,0.293314,0.086033,0.530941,4.473346,0.981939,7.460000e-155
1,Image2,0,8.749969,805.957634,28.389393,0.266538,3.718116,14.464618,63.858816,0.475051,0.225674,0.651352,3.220072,0.988834,7.460000e-155
2,Image3,1,7.341095,1143.808219,33.820234,0.001467,5.061750,26.479563,81.867206,0.031917,0.001019,0.268275,5.981800,0.978014,7.460000e-155
3,Image4,1,5.958145,959.711985,30.979219,0.001477,5.677977,33.428845,151.229741,0.032024,0.001026,0.243851,7.700919,0.964189,7.460000e-155
4,Image5,0,7.315231,729.540579,27.010009,0.146761,4.283221,19.079108,174.988756,0.343849,0.118232,0.501140,6.834689,0.972789,7.460000e-155


In [ ]:
images = data.Image
classes = data.Class
image_label_dict = data.set_index('Image')['Class'].to_dict()
print(image_label_dict)

{'Image1': 0, 'Image2': 0, 'Image3': 1, 'Image4': 1, 'Image5': 0, 'Image6': 0, 'Image7': 0, 'Image8': 0, 'Image9': 0, 'Image10': 1, 'Image11': 1, 'Image12': 1, 'Image13': 1, 'Image14': 0, 'Image15': 0, 'Image16': 1, 'Image17': 1, 'Image18': 0, 'Image19': 0, 'Image20': 0, 'Image21': 0, 'Image22': 0, 'Image23': 0, 'Image24': 0, 'Image25': 0, 'Image26': 1, 'Image27': 0, 'Image28': 0, 'Image29': 0, 'Image30': 0, 'Image31': 0, 'Image32': 1, 'Image33': 1, 'Image34': 1, 'Image35': 0, 'Image36': 0, 'Image37': 1, 'Image38': 0, 'Image39': 0, 'Image40': 0, 'Image41': 0, 'Image42': 0, 'Image43': 0, 'Image44': 0, 'Image45': 0, 'Image46': 0, 'Image47': 0, 'Image48': 0, 'Image49': 0, 'Image50': 0, 'Image51': 0, 'Image52': 1, 'Image53': 1, 'Image54': 1, 'Image55': 1, 'Image56': 0, 'Image57': 1, 'Image58': 1, 'Image59': 1, 'Image60': 1, 'Image61': 0, 'Image62': 0, 'Image63': 0, 'Image64': 0, 'Image65': 0, 'Image66': 0, 'Image67': 0, 'Image68': 0, 'Image69': 1, 'Image70': 0, 'Image71': 0, 'Image72': 0, 

In [ ]:
Y = []
list_filename = image_label_dict.keys()
for filename in list_filename:
  Y.append(image_label_dict[filename])

In [ ]:
import cv2
import numpy as np
from sklearn.decomposition import NMF
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from google.colab.patches import cv2_imshow


def apply_thresholding(image, threshold_value):
    _, thresholded = cv2.threshold(image, threshold_value, 255, cv2.THRESH_BINARY)
    return thresholded

In [ ]:
import numpy as np
import cv2
from os import listdir
from collections import defaultdict
from sklearn.utils import resample

dir_list = '/content/gdrive/MyDrive/NCKH/Crop/'
image_paths = {}
for filename in listdir(dir_list):
    if filename.endswith('.jpg'):
        label = image_label_dict.get(filename.split('.')[0], None)
        if label is not None:
            if label not in image_paths:
                image_paths[label] = []
            image_paths[label].append(dir_list + filename)

# Quy định số lượng mẫu mong muốn trong mỗi lớp
desired_samples = 750

X_balanced = []
Y_balanced = []
X = []
Y = []

# Lấy mẫu ngẫu nhiên từ mỗi lớp với số lượng mẫu mong muốn
for label, paths in image_paths.items():
    sampled_paths = resample(paths, replace=False, n_samples=desired_samples)
    for path in paths:
        image = cv2.imread(path)
        mri_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        thresholded_image = apply_thresholding(mri_image, threshold_value=50)
        image = cv2.bitwise_and(mri_image, mri_image, mask=thresholded_image)
        if image is not None:
            image = cv2.resize(image, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
            # cv2_imshow(image)
            image = image / 255.0
            X_balanced.append(image)
            Y_balanced.append(label)

X = np.array(X_balanced)
Y = np.array(Y_balanced)

In [ ]:
from google.colab.patches import cv2_imshow
# image_paths[label][0]
image = cv2.imread(image_paths[label][100])
mri_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
thresholded_image = apply_thresholding(mri_image, threshold_value=50)
image1 = cv2.bitwise_and(image, image, mask=thresholded_image)
image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
cv2_imshow(image1)
cv2_imshow(image)
cv2_imshow(thresholded_image)
cv2_imshow(mri_image)
image1 = image1.reshape(-1,)
print(image.shape, image1.shape, mri_image.shape)
print(image1)
image1 = image1 / 255.0
unique_values, counts = np.unique(image1, return_counts=True)

# In kết quả
for value, count in zip(unique_values, counts):
    print(f'Giá trị {value} xuất hiện {count} lần')

KeyError: None

In [ ]:
X = np.array(X)
Y = np.array(Y)


In [ ]:
unique_values, counts = np.unique(X[0], return_counts=True)

# In kết quả
for value, count in zip(unique_values, counts):
    print(f'Giá trị {value} xuất hiện {count} lần')

Giá trị 0.0 xuất hiện 10048 lần
Giá trị 0.00392156862745098 xuất hiện 186 lần
Giá trị 0.00784313725490196 xuất hiện 47 lần
Giá trị 0.011764705882352941 xuất hiện 22 lần
Giá trị 0.01568627450980392 xuất hiện 15 lần
Giá trị 0.0196078431372549 xuất hiện 18 lần
Giá trị 0.023529411764705882 xuất hiện 25 lần
Giá trị 0.027450980392156862 xuất hiện 21 lần
Giá trị 0.03137254901960784 xuất hiện 20 lần
Giá trị 0.03529411764705882 xuất hiện 23 lần
Giá trị 0.0392156862745098 xuất hiện 27 lần
Giá trị 0.043137254901960784 xuất hiện 18 lần
Giá trị 0.047058823529411764 xuất hiện 14 lần
Giá trị 0.050980392156862744 xuất hiện 23 lần
Giá trị 0.054901960784313725 xuất hiện 26 lần
Giá trị 0.058823529411764705 xuất hiện 19 lần
Giá trị 0.06274509803921569 xuất hiện 20 lần
Giá trị 0.06666666666666667 xuất hiện 14 lần
Giá trị 0.07058823529411765 xuất hiện 24 lần
Giá trị 0.07450980392156863 xuất hiện 9 lần
Giá trị 0.0784313725490196 xuất hiện 18 lần
Giá trị 0.08235294117647059 xuất hiện 9 lần
Giá trị 0.086274509

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import cv2
import imutils
import numpy as np
import matplotlib.pyplot as plt
import time
from os import listdir

%matplotlib inline

In [ ]:
# n = 500
# for label in [0,1]:
#   images = X[np.argwhere(Y == label)]
#   n_images = images[:n]
#   columns_n = 10
#   rows_n = int(n/ columns_n)
#   plt.figure(figsize=(20, 10))
#   for image in n_images:
#     plt.subplot(rows_n, columns_n, i)
#     plt.imshow(image[0])
#     plt.tick_params(axis='both', which='both', top=False, bottom=False, left=False, right=False,labelbottom=False, labeltop=False, labelleft=False, labelright=False)
#   label_to_str = lambda label: "Yes" if label == 1 else "No"
#   plt.suptitle(f"Brain Tumor: {label_to_str(label)}")
#   plt.show()


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test_val, Y_train, Y_test_val = train_test_split(X, Y, test_size=0.4)
X_test, X_val, Y_test, Y_val = train_test_split(X_test_val, Y_test_val, test_size=0.5)


print ("number of training examples = " + str(X_train.shape[0]))
print ("number of development examples = " + str(X_val.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_val (dev) shape: " + str(X_val.shape))
print ("Y_val (dev) shape: " + str(Y_val.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))


number of training examples = 2257
number of development examples = 753
number of test examples = 752
X_train shape: (2257, 224, 224)
Y_train shape: (2257,)
X_val (dev) shape: (753, 224, 224)
Y_val (dev) shape: (753,)
X_test shape: (752, 224, 224)
Y_test shape: (752,)


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
def build_model(input_shape):
    """
    Arguments:
        input_shape: A tuple representing the shape of the input of the model. shape=(time_steps, #_channels)
    Returns:
        model: A Model object.
    """
    X_input = Input(input_shape)
    # Adjust the architecture for 1D input
    X = Conv2D(32, (1, 7), strides=(1, 1), name='conv0')(X_input)
    X = BatchNormalization(axis=3, name='bn0')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((1, 4), name='max_pool0')(X)
    X = MaxPooling2D((1, 4), name='max_pool1')(X)
    X = Flatten()(X)
    X = Dense(1, activation='sigmoid', name='fc')(X)

    model = Model(inputs=X_input, outputs=X, name='BrainDetectionModel')
    return model

# Create the model with input shape (512, 1)
IMG_SHAPE = (512, 1)
model = build_model(IMG_SHAPE)
model.summary()



ValueError: Input 0 of layer "conv0" is incompatible with the layer: expected min_ndim=4, found ndim=3. Full shape received: (None, 512, 1)

In [ ]:
# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return f"{h}:{m}:{round(s,1)}"
def compute_f1_score(y_true, prob):
    # convert the vector of probabilities to a target vector
    y_pred = np.where(prob > 0.5, 1, 0)

    score = f1_score(y_true, y_pred)

    return score

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, BatchNormalization, Activation, Dropout
from tensorflow.keras.models import Model

def create_cnn_model(input_shape, num_filters):
    inputs = Input(shape=input_shape)
    x = Conv2D(num_filters, (3, 3), activation='relu')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(num_filters * 2, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    outputs = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model1 = create_cnn_model((224, 224, 1), 32)
model2 = create_cnn_model((224, 224, 1), 64)
model3 = create_cnn_model((224, 224, 1), 48)

model1.fit(X_train, Y_train, epochs=8, batch_size=32, validation_split=0.2)
model2.fit(X_train, Y_train, epochs=8, batch_size=32, validation_split=0.2)
model3.fit(X_train, Y_train, epochs=8, batch_size=32, validation_split=0.2)

Epoch 1/8
57/57 [==============================] - 166s 3s/step - loss: 0.6436 - accuracy: 0.7319 - val_loss: 0.3583 - val_accuracy: 0.8628
Epoch 2/8
57/57 [==============================] - 148s 3s/step - loss: 0.3468 - accuracy: 0.8571 - val_loss: 0.4453 - val_accuracy: 0.8031
Epoch 3/8
57/57 [==============================] - 162s 3s/step - loss: 0.2652 - accuracy: 0.8936 - val_loss: 0.2522 - val_accuracy: 0.8982
Epoch 4/8
57/57 [==============================] - 156s 3s/step - loss: 0.2118 - accuracy: 0.9169 - val_loss: 0.2467 - val_accuracy: 0.9071
Epoch 5/8
57/57 [==============================] - 152s 3s/step - loss: 0.1114 - accuracy: 0.9673 - val_loss: 0.2745 - val_accuracy: 0.8761
Epoch 6/8
57/57 [==============================] - 169s 3s/step - loss: 0.0755 - accuracy: 0.9723 - val_loss: 0.2599 - val_accuracy: 0.8916
Epoch 7/8
57/57 [==============================] - 150s 3s/step - loss: 0.0405 - accuracy: 0.9895 - val_loss: 0.2189 - val_accuracy: 0.9248
Epoch 8/8
57/57 [===

In [ ]:
model3.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 224, 224, 1)]     0         
                                                                 
 conv2d_4 (Conv2D)           (None, 222, 222, 48)      480       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 111, 111, 48)      0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 109, 109, 96)      41568     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 54, 54, 96)        0         
 g2D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 279936)            0   

In [ ]:
y_pred1 = model3.predict(X_test)
y_pred2 = (y_pred1 > 0.2).astype(int)
f1 = f1_score(Y_test, y_pred2)
print(f1)
print(f'F1 Score: {f1}')
for i, j in zip(y_pred1, Y_test):
  print(i, j)

24/24 [==============================] - 29s 1s/step
0.8664772727272727
F1 Score: 0.8664772727272727
[9.584151e-06] 0
[8.782002e-08] 0
[0.99997926] 1
[1.35657765e-05] 0
[0.9999999] 1
[2.3177796e-05] 0
[0.9575395] 1
[0.9999925] 1
[0.98747087] 0
[2.9936915e-09] 0
[1.] 1
[8.419092e-09] 0
[0.00026844] 0
[0.99998206] 1
[1.6268813e-06] 0
[8.406985e-05] 0
[0.9840221] 0
[0.9999997] 1
[3.428129e-07] 0
[0.99998116] 0
[0.01489815] 0
[0.00018879] 0
[0.10444482] 0
[0.9997337] 1
[0.00037577] 0
[0.9999915] 1
[0.9941683] 1
[0.99990815] 1
[0.00020651] 0
[0.9999473] 1
[0.00012508] 0
[0.99931425] 1
[1.9106914e-05] 0
[0.9989177] 0
[0.9999992] 1
[2.7308558e-09] 0
[0.99994886] 0
[0.9999967] 1
[0.00348993] 0
[0.9999857] 1
[0.0009857] 1
[3.601406e-07] 0
[6.490063e-07] 0
[7.813421e-05] 0
[0.9998983] 1
[0.01198451] 0
[0.99594814] 1
[0.97280353] 1
[0.9999609] 1
[0.8102512] 1
[0.9551774] 1
[0.9996384] 1
[0.99762976] 1
[2.1373306e-07] 0
[0.9999859] 1
[0.9999619] 1
[0.9993445] 1
[0.29342407] 1
[0.00050185] 0
[1.] 1

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred2))

              precision    recall  f1-score   support

           0       0.95      0.82      0.88       428
           1       0.80      0.94      0.87       324

    accuracy                           0.88       752
   macro avg       0.88      0.88      0.87       752
weighted avg       0.89      0.88      0.88       752



In [ ]:
np.unique(Y_train, return_counts=True)

(array([0, 1]), array([1223, 1034]))

In [ ]:
from tensorflow.keras.layers import concatenate
from tensorflow.keras.models import Model

def create_ensemble_model(models, input_shape):
    # Create a list to hold all the model outputs
    outputs = []
    # Create a common input layer
    common_input = Input(shape=input_shape)

    # Loop through all models and connect the common input to each
    for i, model in enumerate(models):
        # We assume that each model has its layers cloned with a new name to avoid name clashes
        for layer in model.layers[1:]:  # skip the input layer of each model
            layer._name = f'model_{i}_{layer.name}'

        # Connect the model to the common input and store the output
        outputs.append(model(common_input))

    # Concatenate all outputs
    if len(outputs) > 1:
        combined_output = concatenate(outputs)
    else:
        combined_output = outputs[0]

    # Add a dense layer to learn from all combined outputs
    y = Dense(64, activation='relu')(combined_output)
    y = Dense(1, activation='sigmoid')(y)

    # Create the final model
    model = Model(inputs=common_input, outputs=y)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Assuming model1, model2, model3 are already created and compiled
ensemble_model = create_ensemble_model([model1, model2, model3], (224, 224, 1))
ensemble_model.summary()


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 224, 224, 1)]        0         []                            
                                                                                                  
 model (Functional)          (None, 1)                    1196288   ['input_5[0][0]']             
                                                          1                                       
                                                                                                  
 model_1 (Functional)        (None, 1)                    2396249   ['input_5[0][0]']             
                                                          7                                       
                                                                                            

In [ ]:
evaluation = ensemble_model.evaluate(X_val, Y_val)
print(f"Loss: {evaluation[0]}, Accuracy: {evaluation[1]}")

24/24 [==============================] - 90s 4s/step - loss: 0.6902 - accuracy: 0.5286
Loss: 0.6901527047157288, Accuracy: 0.5285524725914001


In [ ]:
# Train the ensemble model
ensemble_model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_val, Y_val))

# Evaluate the model
performance = ensemble_model.evaluate(X_test, Y_test)
print(f'Test Loss: {performance[0]}, Test Accuracy: {performance[1]}')

Epoch 1/10
71/71 [==============================] - 1161s 16s/step - loss: 0.5468 - accuracy: 0.9269 - val_loss: 0.5139 - val_accuracy: 0.8778
Epoch 2/10
71/71 [==============================] - 1114s 16s/step - loss: 0.3527 - accuracy: 0.9796 - val_loss: 0.4087 - val_accuracy: 0.8738
Epoch 3/10
71/71 [==============================] - 1033s 15s/step - loss: 0.2185 - accuracy: 0.9836 - val_loss: 0.3397 - val_accuracy: 0.8725
Epoch 4/10
71/71 [==============================] - 1085s 15s/step - loss: 0.1481 - accuracy: 0.9765 - val_loss: 0.2983 - val_accuracy: 0.8752
Epoch 5/10
71/71 [==============================] - 1099s 15s/step - loss: 0.0890 - accuracy: 0.9903 - val_loss: 0.3215 - val_accuracy: 0.8818
Epoch 6/10
71/71 [==============================] - 1100s 16s/step - loss: 0.0568 - accuracy: 0.9925 - val_loss: 0.3249 - val_accuracy: 0.8898
Epoch 7/10
71/71 [==============================] - 1095s 15s/step - loss: 0.0426 - accuracy: 0.9938 - val_loss: 0.3896 - val_accuracy: 0.8672

In [ ]:
# Predict and calculate F1 score
y_pred = ensemble_model.predict(X_test)
for i, j in zip(y_pred, Y_test):
  print(i, j)
y_pred = (y_pred>0.5 ).astype(int)
for i, j in zip(y_pred, Y_test):
  print(i, j)
f1 = f1_score(Y_test, y_pred)
print(f'F1 Score: {f1}')

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred))

In [ ]:
# def plot_metrics(history):

#     train_loss = history['loss']
#     val_loss = history['val_loss']
#     train_acc = history['accuracy']
#     val_acc = history['val_accuracy']

#     # Loss
#     plt.figure()
#     plt.plot(train_loss, label='Training Loss')
#     plt.plot(val_loss, label='Validation Loss')
#     plt.title('Loss')
#     plt.legend()
#     plt.show()

#     # Accuracy
#     plt.figure()
#     plt.plot(train_acc, label='Training Accuracy')
#     plt.plot(val_acc, label='Validation Accuracy')
#     plt.title('Accuracy')
#     plt.legend()
#     plt.show()

In [ ]:
def plot_metrics(history):
    # Adjust the keys according to what is printed by the `print` statement above
    train_loss = history.get('loss', [])
    val_loss = history.get('val_loss', [])
    train_acc = history.get('accuracy', history.get('acc', []))
    val_acc = history.get('val_accuracy', history.get('val_acc', []))

    if not train_loss or not train_acc:
        print("Loss or accuracy metrics are missing in history.")
        return

    # Loss
    plt.figure()
    plt.plot(train_loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.title('Loss')
    plt.legend()
    plt.show()

    # Accuracy
    plt.figure()
    plt.plot(train_acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.title('Accuracy')
    plt.legend()
    plt.show()

# Gọi hàm với đối số là ensemble_model.history.history
plot_metrics(ensemble_model.history.history)

#Resnet18

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = transform(image)
    image = image.unsqueeze(0)
    return image.to(device)

In [ ]:
def extract_features(image, model):
    with torch.no_grad():
        feature = model(image).squeeze().cpu().numpy()
    return feature

In [ ]:
def extract_features_from_image(image_path, model):
    image = preprocess_image(image_path)
    feature = extract_features(image, model)
    return feature

In [ ]:
resnet18_model = resnet18(pretrained=True)
resnet18_model = nn.Sequential(*list(resnet18_model.children())[:-1])
resnet18_model = resnet18_model.to(device)
resnet18_model.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 118MB/s]


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

In [ ]:
import numpy as np
import cv2
from os import listdir
from collections import defaultdict
from sklearn.utils import resample
X=[]
Y=[]
dir_list = '/content/gdrive/MyDrive/NCKH/Crop/'
image_paths = {}
for filename in tqdm(listdir(dir_list)):
    if filename.endswith('.jpg'):
      path = dir_list + filename
      label = image_label_dict.get(filename.split('.')[0], None)
      X.append(extract_features_from_image(path, resnet18_model))
      Y.append(label)



100%|██████████| 3762/3762 [00:00<00:00, 287690.70it/s]


In [ ]:
df_X = pd.DataFrame(X)
df_X.to_csv(f"/content/gdrive/MyDrive/NCKH/feat_resnet.csv")

In [ ]:
df = pd.read_csv("/content/gdrive/MyDrive/NCKH/feat_resnet.csv")
df_np = df.to_numpy()
X = np.delete(df_np, 0, axis=1)
X[0].shape

(32,)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
clf_svm = SVC(C=10, kernel='rbf', gamma='scale', random_state=42)
clf_svm.fit(X_train, y_train)
preds_svm_res = clf_svm.predict(X_test)
acc_svm = accuracy_score(y_test, preds_svm_res)
conf_mat_svm = confusion_matrix(y_test, preds_svm_res)
print(classification_report(y_test, preds_svm_res))

              precision    recall  f1-score   support

           0       0.88      0.90      0.89       391
           1       0.89      0.86      0.88       362

    accuracy                           0.88       753
   macro avg       0.88      0.88      0.88       753
weighted avg       0.88      0.88      0.88       753



In [ ]:
acc_svm

0.8818061088977424

In [ ]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout


def create_cnn_model(input_shape, num_filters):

  model = Sequential()

  model.add(Conv1D(filters=num_filters, kernel_size=3, activation='relu', input_shape=(512, 1)))
  model.add(MaxPooling1D(pool_size=2))

  model.add(Conv1D(filters=num_filters * 2, kernel_size=3, activation='relu'))
  model.add(MaxPooling1D(pool_size=2))

  model.add(Flatten())

  model.add(Dense(64, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model

model1 = create_cnn_model((512), 32)
model2 = create_cnn_model((512), 64)
model3 = create_cnn_model((512), 48)

model1.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))
model2.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))
model3.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

loss, accuracy = model1.evaluate(X_test, y_test)
print(f'Accuracy1: {accuracy*100:.2f}%')

loss, accuracy = model2.evaluate(X_test, y_test)
print(f'Accuracy2: {accuracy*100:.2f}%')

loss, accuracy = model3.evaluate(X_test, y_test)
print(f'Accuracy3: {accuracy*100:.2f}%')



Epoch 1/10
76/76 [==============================] - 9s 80ms/step - loss: 0.3930 - accuracy: 0.8313 - val_loss: 0.3625 - val_accuracy: 0.8505
Epoch 2/10
76/76 [==============================] - 8s 112ms/step - loss: 0.2703 - accuracy: 0.8974 - val_loss: 0.2792 - val_accuracy: 0.8920
Epoch 3/10
76/76 [==============================] - 3s 42ms/step - loss: 0.2167 - accuracy: 0.9144 - val_loss: 0.2183 - val_accuracy: 0.9302
Epoch 4/10
76/76 [==============================] - 3s 41ms/step - loss: 0.1611 - accuracy: 0.9360 - val_loss: 0.3453 - val_accuracy: 0.8688
Epoch 5/10
76/76 [==============================] - 4s 47ms/step - loss: 0.1428 - accuracy: 0.9427 - val_loss: 0.2016 - val_accuracy: 0.9319
Epoch 6/10
76/76 [==============================] - 6s 85ms/step - loss: 0.1124 - accuracy: 0.9580 - val_loss: 0.2104 - val_accuracy: 0.9153
Epoch 7/10
76/76 [==============================] - 8s 108ms/step - loss: 0.0880 - accuracy: 0.9709 - val_loss: 0.2249 - val_accuracy: 0.9252
Epoch 8/10


In [ ]:
y_pred1 = model1.predict(X_test)

thresholds = np.arange(0, 1.01, 0.01)
best_threshold = 0.0
best_f1 = 0.0

# Thử nghiệm từng ngưỡng và tính toán F1-score
for threshold in thresholds:
    y_pred_classes = (y_pred1 > threshold).astype(int)
    f1 = f1_score(y_test, y_pred_classes)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold

print(f'F1-score1: {best_f1:.4f}')
y_pred1 = (y_pred1 > best_threshold).astype(int)

# for i, j in zip(y_pred1, y_test):
#   print(i, j)

24/24 [==============================] - 1s 24ms/step
F1-score1: 0.9121


In [ ]:
y_pred2 = model2.predict(X_test)

thresholds = np.arange(0, 1.01, 0.01)
best_threshold = 0.0
best_f1 = 0.0

# Thử nghiệm từng ngưỡng và tính toán F1-score
for threshold in thresholds:
    y_pred_classes = (y_pred2 > threshold).astype(int)
    f1 = f1_score(y_test, y_pred_classes)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold

print(f'F1-score2: {best_f1:.4f}')
y_pred2 = (y_pred2 > best_threshold).astype(int)

# for i, j in zip(y_pred2, y_test):
#   print(i, j)

24/24 [==============================] - 1s 33ms/step
F1-score2: 0.9254


In [ ]:
y_pred3 = model3.predict(X_test)

thresholds = np.arange(0, 1.01, 0.01)
best_threshold = 0.0
best_f1 = 0.0

# Thử nghiệm từng ngưỡng và tính toán F1-score
for threshold in thresholds:
    y_pred_classes = (y_pred3 > threshold).astype(int)
    f1 = f1_score(y_test, y_pred_classes)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold

print(f'F1-score3: {best_f1:.4f}')
y_pred3 = (y_pred3 > best_threshold).astype(int)
# for i, j in zip(y_pred3, y_test):
#   print(i, j)

24/24 [==============================] - 1s 28ms/step
F1-score3: 0.9239


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred3))

              precision    recall  f1-score   support

           0       0.94      0.91      0.93       391
           1       0.91      0.94      0.92       362

    accuracy                           0.93       753
   macro avg       0.93      0.93      0.93       753
weighted avg       0.93      0.93      0.93       753



In [ ]:
np.unique(Y_train, return_counts=True)

(array([0, 1]), array([1223, 1034]))

In [ ]:
from tensorflow.keras.layers import concatenate
from tensorflow.keras.models import Model

def create_ensemble_model(models, input_shape):
    # Create a list to hold all the model outputs
    outputs = []
    # Create a common input layer
    common_input = Input(shape=input_shape)

    # Loop through all models and connect the common input to each
    for i, model in enumerate(models):
        # We assume that each model has its layers cloned with a new name to avoid name clashes
        for layer in model.layers[1:]:  # skip the input layer of each model
            layer._name = f'model_{i}_{layer.name}'

        # Connect the model to the common input and store the output
        outputs.append(model(common_input))

    # Concatenate all outputs
    if len(outputs) > 1:
        combined_output = concatenate(outputs)
    else:
        combined_output = outputs[0]

    # Add a dense layer to learn from all combined outputs
    y = Dense(64, activation='relu')(combined_output)
    y = Dense(1, activation='sigmoid')(y)

    # Create the final model
    model = Model(inputs=common_input, outputs=y)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Assuming model1, model2, model3 are already created and compiled
ensemble_model = create_ensemble_model([model1, model2, model3], (512,))
ensemble_model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 512)]                0         []                            
                                                                                                  
 sequential_25 (Sequential)  (None, 1)                    522561    ['input_4[0][0]']             
                                                                                                  
 sequential_26 (Sequential)  (None, 1)                    1057281   ['input_4[0][0]']             
                                                                                                  
 sequential_27 (Sequential)  (None, 1)                    788385    ['input_4[0][0]']             
                                                                                              

In [ ]:
evaluation = ensemble_model.evaluate(X_val, y_val)
print(f"Loss: {evaluation[0]}, Accuracy: {evaluation[1]}")

19/19 [==============================] - 3s 70ms/step - loss: 0.7128 - accuracy: 0.1063
Loss: 0.7127753496170044, Accuracy: 0.10631228983402252


In [ ]:
# Train the ensemble model
ensemble_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model
performance = ensemble_model.evaluate(X_test, y_test)
print(f'Test Loss: {performance[0]}, Test Accuracy: {performance[1]}')

24/24 [==============================] - 2s 81ms/step - loss: 0.2644 - accuracy: 0.9216
Test Loss: 0.26442739367485046, Test Accuracy: 0.921646773815155


In [ ]:
y_pred = ensemble_model.predict(X_test)

thresholds = np.arange(0, 1.01, 0.01)
best_threshold = 0.0
best_f1 = 0.0

# Thử nghiệm từng ngưỡng và tính toán F1-score
for threshold in thresholds:
    y_pred_classes = (y_pred > threshold).astype(int)
    f1 = f1_score(y_test, y_pred_classes)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold

print(f'F1-score: {best_f1:.4f}')
y_pred = (y_pred > best_threshold).astype(int)
# for i, j in zip(y_pred3, y_test):
#   print(i, j)

24/24 [==============================] - 2s 50ms/step
F1-score: 0.9290


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.92      0.93       391
           1       0.92      0.94      0.93       362

    accuracy                           0.93       753
   macro avg       0.93      0.93      0.93       753
weighted avg       0.93      0.93      0.93       753



# NMF

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
from sklearn.decomposition import NMF
import numpy as np
from os import listdir
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
    image = transform(image)
    image = image.unsqueeze(0)
    return image.to(device)

dir_list = '/content/gdrive/MyDrive/NCKH/Crop/'
image_label_dict = {}

n_components = 32
nmf_model = NMF(n_components=n_components, random_state=42)
X_train_nmf = []

for filename in tqdm(listdir(dir_list)):
    if filename.endswith('.jpg'):
        path = dir_list + filename
        image_tensor = preprocess_image(path)
        X_train_nmf.append(image_tensor.squeeze().cpu().numpy().reshape(-1))

X_train_nmf = np.array(X_train_nmf)


X_train_nmf = np.maximum(X_train_nmf, 0)

nmf_model.fit(X_train_nmf)

def extract_features_from_image(image_path, nmf_model):
    image_tensor = preprocess_image(image_path)
    image_array = image_tensor.squeeze().cpu().numpy().reshape(1, -1)
    image_array = np.maximum(image_array, 0)
    features = nmf_model.transform(image_array)
    return features.squeeze()

X = []
Y = []

for filename in tqdm(listdir(dir_list)):
    if filename.endswith('.jpg'):
        path = dir_list + filename
        label = image_label_dict.get(filename.split('.')[0], None)
        feature = extract_features_from_image(path, nmf_model)
        X.append(feature)
        Y.append(label)

X = np.array(X)
Y = np.array(Y)

100%|██████████| 3762/3762 [03:00<00:00, 20.89it/s] 
/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
 99%|█████████▊| 3708/3762 [1:04:36<00:52,  1.03it/s]

In [ ]:
Y=[]
dir_list = '/content/gdrive/MyDrive/NCKH/Crop/'
image_paths = {}
for filename in tqdm(listdir(dir_list)):
    if filename.endswith('.jpg'):
      path = dir_list + filename
      label = image_label_dict.get(filename.split('.')[0], None)
      # X.append(extract_features_from_image(path, resnet18_model))
      Y.append(label)
Y = np.array(Y)
Y

100%|██████████| 3762/3762 [00:00<00:00, 339522.56it/s]


array([1, 0, 1, ..., 1, 1, 0])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Train the SVM model
svm_model = SVC(C=10, kernel='rbf', gamma='scale', random_state=42)
svm_model.fit(X_train, y_train)

# Predict on the test data
y_pred = svm_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)

Accuracy: 0.8924302788844621
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.91      0.90       391
           1       0.90      0.87      0.89       362

    accuracy                           0.89       753
   macro avg       0.89      0.89      0.89       753
weighted avg       0.89      0.89      0.89       753



In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, random_state = 42)


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
clf_svm = SVC(C=10, kernel='rbf', gamma='scale', random_state=42)
clf_svm.fit(X_train, y_train)
preds_svm_res = clf_svm.predict(X_test)
acc_svm = accuracy_score(y_test, preds_svm_res)
conf_mat_svm = confusion_matrix(y_test, preds_svm_res)
print(classification_report(y_test, preds_svm_res))

              precision    recall  f1-score   support

           0       0.88      0.90      0.89       391
           1       0.89      0.86      0.88       362

    accuracy                           0.88       753
   macro avg       0.88      0.88      0.88       753
weighted avg       0.88      0.88      0.88       753



In [ ]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout


def create_cnn_model(input_shape, num_filters):

  model = Sequential()

  model.add(Conv1D(filters=num_filters, kernel_size=3, activation='relu', input_shape=(32, 1)))
  model.add(MaxPooling1D(pool_size=2))

  model.add(Conv1D(filters=num_filters * 2, kernel_size=3, activation='relu'))
  model.add(MaxPooling1D(pool_size=2))

  model.add(Flatten())

  model.add(Dense(64, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model

model1 = create_cnn_model((32), 32)
model2 = create_cnn_model((32), 64)
model3 = create_cnn_model((32), 48)

model1.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))
model2.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))
model3.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

loss, accuracy = model1.evaluate(X_test, y_test)
print(f'Accuracy1: {accuracy*100:.2f}%')

loss, accuracy = model2.evaluate(X_test, y_test)
print(f'Accuracy2: {accuracy*100:.2f}%')

loss, accuracy = model3.evaluate(X_test, y_test)
print(f'Accuracy3: {accuracy*100:.2f}%')



Epoch 1/50
76/76 [==============================] - 3s 14ms/step - loss: 0.6729 - accuracy: 0.5613 - val_loss: 0.6551 - val_accuracy: 0.5698
Epoch 2/50
76/76 [==============================] - 1s 10ms/step - loss: 0.6174 - accuracy: 0.6415 - val_loss: 0.5720 - val_accuracy: 0.7226
Epoch 3/50
76/76 [==============================] - 1s 10ms/step - loss: 0.5470 - accuracy: 0.7349 - val_loss: 0.5166 - val_accuracy: 0.7608
Epoch 4/50
76/76 [==============================] - 0s 6ms/step - loss: 0.4913 - accuracy: 0.7815 - val_loss: 0.4508 - val_accuracy: 0.8189
Epoch 5/50
76/76 [==============================] - 0s 6ms/step - loss: 0.4697 - accuracy: 0.7873 - val_loss: 0.4561 - val_accuracy: 0.8007
Epoch 6/50
76/76 [==============================] - 0s 6ms/step - loss: 0.4593 - accuracy: 0.8006 - val_loss: 0.4538 - val_accuracy: 0.7924
Epoch 7/50
76/76 [==============================] - 0s 6ms/step - loss: 0.4675 - accuracy: 0.7840 - val_loss: 0.4487 - val_accuracy: 0.8007
Epoch 8/50
76/76 

In [ ]:
y_pred1 = model1.predict(X_test)

thresholds = np.arange(0, 1.01, 0.01)
best_threshold = 0.0
best_f1 = 0.0

# Thử nghiệm từng ngưỡng và tính toán F1-score
for threshold in thresholds:
    y_pred_classes = (y_pred1 > threshold).astype(int)
    f1 = f1_score(y_test, y_pred_classes)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold

print(f'F1-score1: {best_f1:.4f}')
y_pred1 = (y_pred1 > best_threshold).astype(int)

# for i, j in zip(y_pred1, y_test):
#   print(i, j)

24/24 [==============================] - 1s 13ms/step
F1-score1: 0.8432


In [ ]:
y_pred2 = model2.predict(X_test)

thresholds = np.arange(0, 1.01, 0.01)
best_threshold = 0.0
best_f1 = 0.0

# Thử nghiệm từng ngưỡng và tính toán F1-score
for threshold in thresholds:
    y_pred_classes = (y_pred2 > threshold).astype(int)
    f1 = f1_score(y_test, y_pred_classes)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold

print(f'F1-score2: {best_f1:.4f}')
y_pred2 = (y_pred2 > best_threshold).astype(int)

# for i, j in zip(y_pred2, y_test):
#   print(i, j)

24/24 [==============================] - 1s 8ms/step
F1-score2: 0.8403


In [ ]:
y_pred3 = model3.predict(X_test)

thresholds = np.arange(0, 1.01, 0.01)
best_threshold = 0.0
best_f1 = 0.0

# Thử nghiệm từng ngưỡng và tính toán F1-score
for threshold in thresholds:
    y_pred_classes = (y_pred3 > threshold).astype(int)
    f1 = f1_score(y_test, y_pred_classes)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold

print(f'F1-score3: {best_f1:.4f}')
y_pred3 = (y_pred3 > best_threshold).astype(int)
# for i, j in zip(y_pred3, y_test):
#   print(i, j)

24/24 [==============================] - 1s 9ms/step
F1-score3: 0.8418


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred3))

              precision    recall  f1-score   support

           0       0.90      0.79      0.84       391
           1       0.80      0.90      0.85       362

    accuracy                           0.84       753
   macro avg       0.85      0.85      0.84       753
weighted avg       0.85      0.84      0.84       753



In [ ]:
np.unique(Y_train, return_counts=True)

NameError: name 'Y_train' is not defined

In [ ]:
from tensorflow.keras.layers import concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense


def create_ensemble_model(models, input_shape):
    # Create a list to hold all the model outputs
    outputs = []
    # Create a common input layer
    common_input = Input(shape=(32,1))

    # Loop through all models and connect the common input to each
    for i, model in enumerate(models):
        # We assume that each model has its layers cloned with a new name to avoid name clashes
        for layer in model.layers[1:]:  # skip the input layer of each model
            layer._name = f'model_{i}_{layer.name}'

        # Connect the model to the common input and store the output
        outputs.append(model(common_input))

    # Concatenate all outputs
    if len(outputs) > 1:
        combined_output = concatenate(outputs)
    else:
        combined_output = outputs[0]

    # Add a dense layer to learn from all combined outputs
    y = Dense(64, activation='relu')(combined_output)
    y = Dense(1, activation='sigmoid')(y)

    # Create the final model
    model = Model(inputs=common_input, outputs=y)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Assuming model1, model2, model3 are already created and compiled
ensemble_model = create_ensemble_model([model1, model2, model3], (32,))
ensemble_model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 1)]              0         []                            
                                                                                                  
 sequential_9 (Sequential)   (None, 1)                    31041     ['input_1[0][0]']             
                                                                                                  
 sequential_10 (Sequential)  (None, 1)                    74241     ['input_1[0][0]']             
                                                                                                  
 sequential_11 (Sequential)  (None, 1)                    51105     ['input_1[0][0]']             
                                                                                              

In [ ]:
evaluation = ensemble_model.evaluate(X_val, y_val)
print(f"Loss: {evaluation[0]}, Accuracy: {evaluation[1]}")

19/19 [==============================] - 1s 7ms/step - loss: 0.6444 - accuracy: 0.4369
Loss: 0.6444294452667236, Accuracy: 0.4368770718574524


In [ ]:
# Train the ensemble model
ensemble_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model
performance = ensemble_model.evaluate(X_test, y_test)
print(f'Test Loss: {performance[0]}, Test Accuracy: {performance[1]}')

Epoch 1/50
76/76 [==============================] - 14s 23ms/step - loss: 0.5484 - accuracy: 0.8492 - val_loss: 0.4728 - val_accuracy: 0.8787
Epoch 2/50
76/76 [==============================] - 2s 24ms/step - loss: 0.4117 - accuracy: 0.8837 - val_loss: 0.3952 - val_accuracy: 0.8621
Epoch 3/50
76/76 [==============================] - 2s 24ms/step - loss: 0.3267 - accuracy: 0.8878 - val_loss: 0.3093 - val_accuracy: 0.8854
Epoch 4/50
76/76 [==============================] - 2s 31ms/step - loss: 0.2964 - accuracy: 0.8853 - val_loss: 0.3213 - val_accuracy: 0.8638
Epoch 5/50
76/76 [==============================] - 2s 29ms/step - loss: 0.2777 - accuracy: 0.8899 - val_loss: 0.2801 - val_accuracy: 0.8953
Epoch 6/50
76/76 [==============================] - 2s 21ms/step - loss: 0.2863 - accuracy: 0.8878 - val_loss: 0.2838 - val_accuracy: 0.8804
Epoch 7/50
76/76 [==============================] - 1s 14ms/step - loss: 0.2592 - accuracy: 0.8953 - val_loss: 0.3144 - val_accuracy: 0.8605
Epoch 8/50
7

In [ ]:
y_pred = ensemble_model.predict(X_test)

thresholds = np.arange(0, 1.01, 0.01)
best_threshold = 0.0
best_f1 = 0.0

# Thử nghiệm từng ngưỡng và tính toán F1-score
for threshold in thresholds:
    y_pred_classes = (y_pred > threshold).astype(int)
    f1 = f1_score(y_test, y_pred_classes)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold

print(f'F1-score: {best_f1:.4f}')
y_pred = (y_pred > best_threshold).astype(int)
# for i, j in zip(y_pred3, y_test):
#   print(i, j)

24/24 [==============================] - 1s 7ms/step
F1-score: 0.9129


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.92      0.93       391
           1       0.92      0.94      0.93       362

    accuracy                           0.93       753
   macro avg       0.93      0.93      0.93       753
weighted avg       0.93      0.93      0.93       753



# Combine NMF + Resnet

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
from torchvision.models import resnet18
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.decomposition import NMF
from sklearn.model_selection import train_test_split

# Kiểm tra nếu có GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hàm tiền xử lý ảnh
def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = transform(image)
    image = image.unsqueeze(0)
    return image.to(device)

# Hàm trích xuất đặc trưng từ ResNet
def extract_features(image, model):
    with torch.no_grad():
        feature = model(image).squeeze().cpu().numpy()
    return feature

def extract_features_from_image(image_path, model):
    image = preprocess_image(image_path)
    feature = extract_features(image, model)
    return feature

# Tải mô hình ResNet18 đã huấn luyện sẵn
resnet18_model = resnet18(pretrained=True)
resnet18_model = nn.Sequential(*list(resnet18_model.children())[:-1])
resnet18_model = resnet18_model.to(device)
resnet18_model.eval()

# Đọc và trích xuất đặc trưng từ ResNet
dir_list = '/content/gdrive/MyDrive/NCKH/Crop/'
images = data.Image
classes = data.Class
image_label_dict = data.set_index('Image')['Class'].to_dict()
print(image_label_dict)
X_resnet = []
Y = []

for filename in tqdm(listdir(dir_list)):
    if filename.endswith('.jpg'):
        path = dir_list + filename
        label = image_label_dict.get(filename.split('.')[0], None)
        if label is not None:
            feature = extract_features_from_image(path, resnet18_model)
            X_resnet.append(feature)
            Y.append(label)

# Chuyển đổi danh sách thành mảng numpy
X_resnet = np.array(X_resnet)

# Áp dụng NMF lên các đặc trưng từ ResNet
n_components = 32
nmf_model = NMF(n_components=n_components, random_state=42)
X_nmf = nmf_model.fit_transform(X_resnet)

# Kết hợp đặc trưng từ ResNet và NMF
X_combined = np.hstack((X_resnet, X_nmf))

# Chia dữ liệu thành tập huấn luyện, tập kiểm tra và tập validation
X_train, X_test, y_train, y_test = train_test_split(X_combined, Y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Huấn luyện mô hình SVM với đặc trưng kết hợp
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# Dự đoán và đánh giá mô hình
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 126MB/s]


{'Image1': 0, 'Image2': 0, 'Image3': 1, 'Image4': 1, 'Image5': 0, 'Image6': 0, 'Image7': 0, 'Image8': 0, 'Image9': 0, 'Image10': 1, 'Image11': 1, 'Image12': 1, 'Image13': 1, 'Image14': 0, 'Image15': 0, 'Image16': 1, 'Image17': 1, 'Image18': 0, 'Image19': 0, 'Image20': 0, 'Image21': 0, 'Image22': 0, 'Image23': 0, 'Image24': 0, 'Image25': 0, 'Image26': 1, 'Image27': 0, 'Image28': 0, 'Image29': 0, 'Image30': 0, 'Image31': 0, 'Image32': 1, 'Image33': 1, 'Image34': 1, 'Image35': 0, 'Image36': 0, 'Image37': 1, 'Image38': 0, 'Image39': 0, 'Image40': 0, 'Image41': 0, 'Image42': 0, 'Image43': 0, 'Image44': 0, 'Image45': 0, 'Image46': 0, 'Image47': 0, 'Image48': 0, 'Image49': 0, 'Image50': 0, 'Image51': 0, 'Image52': 1, 'Image53': 1, 'Image54': 1, 'Image55': 1, 'Image56': 0, 'Image57': 1, 'Image58': 1, 'Image59': 1, 'Image60': 1, 'Image61': 0, 'Image62': 0, 'Image63': 0, 'Image64': 0, 'Image65': 0, 'Image66': 0, 'Image67': 0, 'Image68': 0, 'Image69': 1, 'Image70': 0, 'Image71': 0, 'Image72': 0, 

100%|██████████| 3762/3762 [07:39<00:00,  8.19it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


Accuracy: 0.9070385126162018
Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.93      0.91       391
           1       0.92      0.89      0.90       362

    accuracy                           0.91       753
   macro avg       0.91      0.91      0.91       753
weighted avg       0.91      0.91      0.91       753



In [ ]:
import torch
from torchvision import transforms
from PIL import Image
from torchvision.models import resnet18
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.decomposition import NMF
from sklearn.model_selection import train_test_split
import torch.nn as nn

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
from torchvision.models import resnet18
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.decomposition import NMF
from sklearn.model_selection import train_test_split
import torch.nn as nn

# Kiểm tra nếu có GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hàm tiền xử lý ảnh
def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = transform(image)
    image = image.unsqueeze(0)
    return image.to(device)

# Hàm trích xuất đặc trưng từ ResNet
def extract_features(image, model):
    with torch.no_grad():
        feature = model(image).squeeze().cpu().numpy()
    return feature

def extract_features_from_image(image_path, model):
    image = preprocess_image(image_path)
    feature = extract_features(image, model)
    return feature

# Tải mô hình ResNet18 đã huấn luyện sẵn
resnet18_model = resnet18(pretrained=True)
resnet18_model = nn.Sequential(*list(resnet18_model.children())[:-1])
resnet18_model = resnet18_model.to(device)
resnet18_model.eval()

# Đọc và trích xuất đặc trưng từ ResNet
dir_list = '/content/gdrive/MyDrive/NCKH/Crop/'
images = data.Image
classes = data.Class
image_label_dict = data.set_index('Image')['Class'].to_dict()
print(image_label_dict)

X_resnet = []
Y = []

for filename in tqdm(listdir(dir_list)):
    if filename.endswith('.jpg'):
        path = dir_list + filename
        label = image_label_dict.get(filename.split('.')[0], None)
        if label is not None:
            feature = extract_features_from_image(path, resnet18_model)
            X_resnet.append(feature)
            Y.append(label)

# Chuyển đổi danh sách thành mảng numpy
X_resnet = np.array(X_resnet)
Y = np.array(Y)

# Áp dụng NMF lên các đặc trưng từ ResNet
n_components = 100
nmf_model = NMF(n_components=n_components, random_state=42)
X_nmf = nmf_model.fit_transform(X_resnet)

# Kết hợp đặc trưng từ ResNet và NMF
X_combined = np.hstack((X_resnet, X_nmf))

# Chuyển đổi thành dạng phù hợp với Conv1D
X_combined = X_combined.reshape((X_combined.shape[0], X_combined.shape[1], 1))
from sklearn.model_selection import train_test_split
X_train, X_test_val, Y_train, Y_test_val = train_test_split(X_combined, Y, test_size=0.4, random_state=42)
X_test, X_val, Y_test, Y_val = train_test_split(X_test_val, Y_test_val, test_size=0.5, random_state=42)
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


{'Image1': 0, 'Image2': 0, 'Image3': 1, 'Image4': 1, 'Image5': 0, 'Image6': 0, 'Image7': 0, 'Image8': 0, 'Image9': 0, 'Image10': 1, 'Image11': 1, 'Image12': 1, 'Image13': 1, 'Image14': 0, 'Image15': 0, 'Image16': 1, 'Image17': 1, 'Image18': 0, 'Image19': 0, 'Image20': 0, 'Image21': 0, 'Image22': 0, 'Image23': 0, 'Image24': 0, 'Image25': 0, 'Image26': 1, 'Image27': 0, 'Image28': 0, 'Image29': 0, 'Image30': 0, 'Image31': 0, 'Image32': 1, 'Image33': 1, 'Image34': 1, 'Image35': 0, 'Image36': 0, 'Image37': 1, 'Image38': 0, 'Image39': 0, 'Image40': 0, 'Image41': 0, 'Image42': 0, 'Image43': 0, 'Image44': 0, 'Image45': 0, 'Image46': 0, 'Image47': 0, 'Image48': 0, 'Image49': 0, 'Image50': 0, 'Image51': 0, 'Image52': 1, 'Image53': 1, 'Image54': 1, 'Image55': 1, 'Image56': 0, 'Image57': 1, 'Image58': 1, 'Image59': 1, 'Image60': 1, 'Image61': 0, 'Image62': 0, 'Image63': 0, 'Image64': 0, 'Image65': 0, 'Image66': 0, 'Image67': 0, 'Image68': 0, 'Image69': 1, 'Image70': 0, 'Image71': 0, 'Image72': 0, 

100%|██████████| 3762/3762 [07:51<00:00,  7.98it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


In [ ]:
Y_train = np.array(Y_train)
Y_val = np.array(Y_val)
Y_test = np.array(Y_test)
X_combined = X_combined.reshape((X_combined.shape[0], X_combined.shape[1], 1))

In [ ]:
def create_cnn_model(input_shape, num_filters):
    model = Sequential()
    model.add(Conv1D(filters=num_filters, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=num_filters * 2, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model
input_shape = (X_combined.shape[1], 1)
model1 = create_cnn_model(input_shape, 32)
model2 = create_cnn_model(input_shape, 64)
model3 = create_cnn_model(input_shape, 48)
model1.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_val, Y_val))
model2.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_val, Y_val))
model3.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_val, Y_val))

loss, accuracy = model1.evaluate(X_test, Y_test)
print(f'Accuracy1: {accuracy*100:.2f}%')

loss, accuracy = model2.evaluate(X_test, Y_test)
print(f'Accuracy2: {accuracy*100:.2f}%')

loss, accuracy = model3.evaluate(X_test, Y_test)
print(f'Accuracy3: {accuracy*100:.2f}%')


Epoch 1/10
71/71 [==============================] - 4s 36ms/step - loss: 0.4595 - accuracy: 0.8033 - val_loss: 0.3427 - val_accuracy: 0.8699
Epoch 2/10
71/71 [==============================] - 2s 34ms/step - loss: 0.2964 - accuracy: 0.8773 - val_loss: 0.2748 - val_accuracy: 0.8898
Epoch 3/10
71/71 [==============================] - 2s 32ms/step - loss: 0.2418 - accuracy: 0.9047 - val_loss: 0.2456 - val_accuracy: 0.9057
Epoch 4/10
71/71 [==============================] - 4s 50ms/step - loss: 0.1925 - accuracy: 0.9216 - val_loss: 0.2356 - val_accuracy: 0.9150
Epoch 5/10
71/71 [==============================] - 3s 43ms/step - loss: 0.1592 - accuracy: 0.9393 - val_loss: 0.2593 - val_accuracy: 0.8977
Epoch 6/10
71/71 [==============================] - 2s 34ms/step - loss: 0.1627 - accuracy: 0.9335 - val_loss: 0.2077 - val_accuracy: 0.9177
Epoch 7/10
71/71 [==============================] - 2s 32ms/step - loss: 0.1352 - accuracy: 0.9446 - val_loss: 0.2011 - val_accuracy: 0.9230
Epoch 8/10
71

In [ ]:
from tensorflow.keras.layers import concatenate, Dense, Input
from tensorflow.keras.models import Model

def create_ensemble_model(models, input_shape):
    # Create a list to hold all the model outputs
    outputs = []

    # Create a common input layer
    common_input = Input(shape=input_shape)

    # Loop through all models and connect the common input to each
    for i, model in enumerate(models):
        # We assume that each model has its layers cloned with a new name to avoid name clashes
        for layer in model.layers[1:]:  # skip the input layer of each model
            layer._name = f'model_{i}_{layer.name}'

        # Connect the model to the common input and store the output
        outputs.append(model(common_input))

    # Concatenate all outputs
    if len(outputs) > 1:
        combined_output = concatenate(outputs)
    else:
        combined_output = outputs[0]

    # Add a dense layer to learn from all combined outputs
    y = Dense(64, activation='relu')(combined_output)
    y = Dense(1, activation='sigmoid')(y)

    # Create the final model
    ensemble_model = Model(inputs=common_input, outputs=y)

    # Compile the model
    ensemble_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return ensemble_model

# Assuming model1, model2, model3 are already created and compiled
ensemble_model = create_ensemble_model([model1, model2, model3], input_shape=(X_combined.shape[1], 1))
ensemble_model.summary()


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_10 (InputLayer)       [(None, 612, 1)]             0         []                            
                                                                                                  
 sequential_33 (Sequential)  (None, 1)                    624961    ['input_10[0][0]']            
                                                                                                  
 sequential_34 (Sequential)  (None, 1)                    1262081   ['input_10[0][0]']            
                                                                                                  
 sequential_35 (Sequential)  (None, 1)                    941985    ['input_10[0][0]']            
                                                                                            

In [ ]:
from sklearn.model_selection import train_test_split

# Chia dữ liệu thành tập huấn luyện và tập validation
X_train, X_test_val, Y_train, Y_test_val = train_test_split(X_combined, Y, test_size=0.4, random_state=42)
X_test, X_val, Y_test, Y_val = train_test_split(X_test_val, Y_test_val, test_size=0.5, random_state=42)

# Chuyển đổi Y_val thành mảng NumPy nếu chưa phải
Y_val = np.array(Y_val)

# Evaluate ensemble model trên dữ liệu validation
evaluation = ensemble_model.evaluate(X_val, Y_val)
print(f"Loss: {evaluation[0]}, Accuracy: {evaluation[1]}")

24/24 [==============================] - 1s 32ms/step - loss: 0.6260 - accuracy: 0.4489
Loss: 0.6259624361991882, Accuracy: 0.4488711953163147


In [ ]:
from sklearn.model_selection import train_test_split

# Split X_test_val into X_val and X_test again
X_test, X_val, Y_test, Y_val = train_test_split(X_test_val, Y_test_val, test_size=0.4)

# Ensure Y_val and Y_test are numpy arrays
Y_val = np.array(Y_val)
Y_test = np.array(Y_test)

# Train the ensemble model
ensemble_model.fit(X_train, Y_train, epochs=30, batch_size=32, validation_data=(X_val, Y_val))

# Evaluate the model
performance = ensemble_model.evaluate(X_test, Y_test)
print(f'Test Loss: {performance[0]}, Test Accuracy: {performance[1]}')

Epoch 1/30
71/71 [==============================] - 29s 171ms/step - loss: 0.5066 - accuracy: 0.9424 - val_loss: 0.4490 - val_accuracy: 0.9236
Epoch 2/30
71/71 [==============================] - 14s 200ms/step - loss: 0.2944 - accuracy: 0.9832 - val_loss: 0.2985 - val_accuracy: 0.9269
Epoch 3/30
71/71 [==============================] - 11s 161ms/step - loss: 0.1605 - accuracy: 0.9845 - val_loss: 0.2517 - val_accuracy: 0.9136
Epoch 4/30
71/71 [==============================] - 11s 152ms/step - loss: 0.1015 - accuracy: 0.9863 - val_loss: 0.2169 - val_accuracy: 0.9252
Epoch 5/30
71/71 [==============================] - 14s 205ms/step - loss: 0.0619 - accuracy: 0.9911 - val_loss: 0.2251 - val_accuracy: 0.9219
Epoch 6/30
71/71 [==============================] - 12s 173ms/step - loss: 0.0470 - accuracy: 0.9929 - val_loss: 0.2080 - val_accuracy: 0.9136
Epoch 7/30
71/71 [==============================] - 12s 171ms/step - loss: 0.0242 - accuracy: 0.9982 - val_loss: 0.2425 - val_accuracy: 0.9203

#Resnet -> NMF

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
from torchvision.models import resnet18
import numpy as np
from tqdm import tqdm

# Kiểm tra nếu có GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hàm tiền xử lý ảnh
def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = transform(image)
    image = image.unsqueeze(0)
    return image.to(device)

# Hàm trích xuất đặc trưng từ ResNet
def extract_features(image, model):
    with torch.no_grad():
        feature = model(image).squeeze().cpu().numpy()
    return feature

# Tải mô hình ResNet18 đã huấn luyện sẵn
resnet18_model = resnet18(pretrained=True)
resnet18_model = torch.nn.Sequential(*list(resnet18_model.children())[:-1])
resnet18_model = resnet18_model.to(device)
resnet18_model.eval()

# Định nghĩa hàm trích xuất đặc trưng từ ảnh
def extract_features_from_image(image_path, model):
    image = preprocess_image(image_path)
    feature = extract_features(image, model)
    return feature


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 129MB/s]


In [ ]:
dir_list = '/content/gdrive/MyDrive/NCKH/Crop/'
images = data.Image
classes = data.Class
image_label_dict = data.set_index('Image')['Class'].to_dict()
print(image_label_dict)
import numpy as np
import cv2
from os import listdir
from collections import defaultdict
from sklearn.utils import resample


{'Image1': 0, 'Image2': 0, 'Image3': 1, 'Image4': 1, 'Image5': 0, 'Image6': 0, 'Image7': 0, 'Image8': 0, 'Image9': 0, 'Image10': 1, 'Image11': 1, 'Image12': 1, 'Image13': 1, 'Image14': 0, 'Image15': 0, 'Image16': 1, 'Image17': 1, 'Image18': 0, 'Image19': 0, 'Image20': 0, 'Image21': 0, 'Image22': 0, 'Image23': 0, 'Image24': 0, 'Image25': 0, 'Image26': 1, 'Image27': 0, 'Image28': 0, 'Image29': 0, 'Image30': 0, 'Image31': 0, 'Image32': 1, 'Image33': 1, 'Image34': 1, 'Image35': 0, 'Image36': 0, 'Image37': 1, 'Image38': 0, 'Image39': 0, 'Image40': 0, 'Image41': 0, 'Image42': 0, 'Image43': 0, 'Image44': 0, 'Image45': 0, 'Image46': 0, 'Image47': 0, 'Image48': 0, 'Image49': 0, 'Image50': 0, 'Image51': 0, 'Image52': 1, 'Image53': 1, 'Image54': 1, 'Image55': 1, 'Image56': 0, 'Image57': 1, 'Image58': 1, 'Image59': 1, 'Image60': 1, 'Image61': 0, 'Image62': 0, 'Image63': 0, 'Image64': 0, 'Image65': 0, 'Image66': 0, 'Image67': 0, 'Image68': 0, 'Image69': 1, 'Image70': 0, 'Image71': 0, 'Image72': 0, 

In [ ]:
from sklearn.decomposition import NMF

# Load và trích xuất đặc trưng từ ResNet cho từng ảnh
X_resnet = []
for filename in tqdm(listdir(dir_list)):
    if filename.endswith('.jpg'):
        path = dir_list + filename
        feature = extract_features_from_image(path, resnet18_model)
        X_resnet.append(feature)

# Chuyển đổi danh sách thành mảng numpy
X_resnet = np.array(X_resnet)

# Áp dụng NMF lên các đặc trưng từ ResNet
n_components = 32
nmf_model = NMF(n_components=n_components, random_state=42)
X_nmf = nmf_model.fit_transform(X_resnet)


100%|██████████| 3762/3762 [10:08<00:00,  6.18it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load và chia dữ liệu thành tập huấn luyện và validation
X_train_val, X_test, y_train_val, y_test = train_test_split(X_nmf, Y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

# Huấn luyện SVM
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, y_train)

# Đánh giá SVM trên tập validation
y_pred = svm_model.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print(f'Accuracy of SVM on validation set: {accuracy * 100:.2f}%')

# Đánh giá SVM trên tập test
y_pred_test = svm_model.predict(X_test)
accuracy_test = accuracy_score(y_test, y_pred_test)
print(f'Accuracy of SVM on test set: {accuracy_test * 100:.2f}%')


Accuracy of SVM on validation set: 56.81%
Accuracy of SVM on test set: 55.64%


In [ ]:
import torch
from torchvision import transforms
from PIL import Image
from torchvision.models import resnet18
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.decomposition import NMF
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Kiểm tra nếu có GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hàm tiền xử lý ảnh
def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = transform(image)
    image = image.unsqueeze(0)
    return image.to(device)

# Hàm trích xuất đặc trưng từ ResNet
def extract_features(image, model):
    with torch.no_grad():
        feature = model(image).squeeze().cpu().numpy()
    return feature

def extract_features_from_image(image_path, model):
    image = preprocess_image(image_path)
    feature = extract_features(image, model)
    return feature

# Tải mô hình ResNet18 đã huấn luyện sẵn
resnet18_model = resnet18(pretrained=True)
resnet18_model = nn.Sequential(*list(resnet18_model.children())[:-1])
resnet18_model = resnet18_model.to(device)
resnet18_model.eval()

# Đọc và trích xuất đặc trưng từ ResNet
dir_list = '/content/gdrive/MyDrive/NCKH/Crop/'
images = data.Image
classes = data.Class
image_label_dict = data.set_index('Image')['Class'].to_dict()
print(image_label_dict)
X_resnet = []
Y = []

for filename in tqdm(listdir(dir_list)):
    if filename.endswith('.jpg'):
        path = dir_list + filename
        label = image_label_dict.get(filename.split('.')[0], None)
        if label is not None:
            feature = extract_features_from_image(path, resnet18_model)
            X_resnet.append(feature)
            Y.append(label)

# Chuyển đổi danh sách thành mảng numpy
X_resnet = np.array(X_resnet)

# Áp dụng NMF lên các đặc trưng từ ResNet
n_components = 32
nmf_model = NMF(n_components=n_components, random_state=42)
X_nmf = nmf_model.fit_transform(X_resnet)

# Kết hợp đặc trưng từ ResNet và NMF
X_combined = np.hstack((X_resnet, X_nmf))

# Chia dữ liệu thành tập huấn luyện, tập kiểm tra và tập validation
X_train, X_test, y_train, y_test = train_test_split(X_combined, Y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Huấn luyện mô hình SVM với đặc trưng kết hợp
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# Dự đoán và đánh giá mô hình
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


{'Image1': 0, 'Image2': 0, 'Image3': 1, 'Image4': 1, 'Image5': 0, 'Image6': 0, 'Image7': 0, 'Image8': 0, 'Image9': 0, 'Image10': 1, 'Image11': 1, 'Image12': 1, 'Image13': 1, 'Image14': 0, 'Image15': 0, 'Image16': 1, 'Image17': 1, 'Image18': 0, 'Image19': 0, 'Image20': 0, 'Image21': 0, 'Image22': 0, 'Image23': 0, 'Image24': 0, 'Image25': 0, 'Image26': 1, 'Image27': 0, 'Image28': 0, 'Image29': 0, 'Image30': 0, 'Image31': 0, 'Image32': 1, 'Image33': 1, 'Image34': 1, 'Image35': 0, 'Image36': 0, 'Image37': 1, 'Image38': 0, 'Image39': 0, 'Image40': 0, 'Image41': 0, 'Image42': 0, 'Image43': 0, 'Image44': 0, 'Image45': 0, 'Image46': 0, 'Image47': 0, 'Image48': 0, 'Image49': 0, 'Image50': 0, 'Image51': 0, 'Image52': 1, 'Image53': 1, 'Image54': 1, 'Image55': 1, 'Image56': 0, 'Image57': 1, 'Image58': 1, 'Image59': 1, 'Image60': 1, 'Image61': 0, 'Image62': 0, 'Image63': 0, 'Image64': 0, 'Image65': 0, 'Image66': 0, 'Image67': 0, 'Image68': 0, 'Image69': 1, 'Image70': 0, 'Image71': 0, 'Image72': 0, 

100%|██████████| 3762/3762 [08:34<00:00,  7.31it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


Accuracy: 0.9070385126162018
Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.93      0.91       391
           1       0.92      0.89      0.90       362

    accuracy                           0.91       753
   macro avg       0.91      0.91      0.91       753
weighted avg       0.91      0.91      0.91       753



In [ ]:
# prompt: performance

print(performance)
